# Hypothesentests

Wenn wir nachweisen wollen, dass ein Unterschied zwischen zwei Verteilungen nicht zufällig entstanden ist, sondern kausal mit einer Variable zusammenhängt, dann führen wir einen Hypothesentest durch.

Ein solcher Test sagt uns, mit welcher Wahrscheinlichkeit ein Unterschied kausal ist und wie groß der Effekt ist. Wir sprechen in dem Zusammenhang von **statistischer Signifikanz** und **Effektgröße**.

Wie immer beginnt alles mit den Daten.

## Laden und Verstehen der Daten
Lies die Daten (`"02_Sales.csv"`) ein und mache dich damit vertraut.
Die Daten basieren auf diesem [Kaggle Link](https://www.kaggle.com/datasets/matinmahmoudi/sales-and-satisfaction/data?select=Sales_without_NaNs_v1.3.csv) und sind sythetisch erzeugt, um damit statistische Methoden zu üben.

In [ ]:
import pandas as pd

df = pd.read_csv("02_Sales.csv")

In [ ]:
df

**Welche Analysen könnten wir mit diesen Daten machen? Versuche selbst ein paar Gedanken zu entwickeln bevor ich gleich meine Gedanken teile**

# Mögliche Analysen
## Korrelationen
- Wie sehr hängt die Zufriedenheit mit den Verkaufszahlen zusammen? → **Pearson's-r**
- Wie sehr hängt die Zufriedenheit bzw. Verkauf vorher mit der Zufriedenheit bzw. Verkauf nachher zusammen? → **Pearson's-r**

## Kausalität
- Sind die Verkaufszahlen oder Zufriedenheit der Treatment Gruppe signifikant unterschiedlich von den Verkaufszahlen bzw. Zufriedenheit in der Kontrollgruppe? → **Independent Samples t-test**
- Sind die Verkaufszahlen bzw. Zufriedenheit in einer Gruppe nachher signifikant anders als vorher? → **Repeated Samples t-test**
- Hat die Treatment Gruppe deutlich mehr Käufe getätigt als die Kontrollgruppe → **Bonus: Chi Square Test**

# Independent Samples t-test
## Wann wenden wir ihn an?
Ein **independent samples t-test** erlaubt uns zu bestimmen wie wahrscheinlich es ist, dass ein Unterschied **zwischen zwei Gruppen** zufällig entstanden ist. Diese Wahrscheinlichkeit wird oft mit $p$ angegeben und bei $p<0.05$ spricht man häufig von **statistischer Signifikanz**.

Das heißt nichts anderes als, dass die Chance dass der Unterschied zufällig entstanden ist, bei weniger als 5% liegt. Im Umkehrschluss ist es 95% wahrscheinlich, dass die zwei Gruppen tatsächlich unterschiedlich sind und der beobachtete Effekt auf die Variable zurückzuführen ist, die die Gruppen unterscheidet (Medizin vs Placebo, Website Design A vs B, Jung vs Alt, ...)

## Wie wenden wir einen **independent samples t-test** an?

Das geht in Python ganz einfach! Wir müssen uns lediglich bewusst sein, was unsere Gruppierungsvariable ist und was unsere Zielvariable ist.

In diesem Fall nehmen wir die Variable `Group` (also Control vs. Treatment) als Gruppierungsvariable und die numerische Variable `Sales_After` als Zielvariable.

Hinweis:
- Wir könnten (und sollten vermutlich) auch die `Sales_Before` Variable untersuchen, um sicherzustellen, dass es nicht schon vor der Intervention einen signifikanten Unterschied zwischen den Gruppen gab. Wenn die Treatment Group zufällig wohlhabendere Kunden beinhaltet als die Control Group, ist es möglich, dass dieser Unterschied sich auch nach der Intervention bemerkbar macht und unsere Schlussfolgerungen verzerrt.
- Wir könnten die Analyse analog für die `Satisfaction` Variable durchführen
- `Purchase_Made` ist eine kategorische (binäre) Variable und eignet sich **nicht** für einen t-test

In [ ]:
from scipy import stats

In [ ]:
# Wir filtern unser DataFrame auf die Datenpunkte der jeweiligen Gruppen und wählen dann die Zielvariable aus
# Ein t-test ist symmetrisch. Es ist also nicht wichtig, in welcher Reihenfolge wir die beiden zu prüfenden Gruppen eingeben.

t_stat, p_value = stats.ttest_ind(df[df.Group == "Control"].Sales_After, df[df.Group == "Treatment"].Sales_After)

print("p-value:", p_value)

In diesem Beispiel ist der $p=0$ was natürlich kleiner ist als 5%. Somit können wir schlussfolgern, dass die Maßnahme, welche die Treatment Gruppe von der Kontrollgruppe unterscheidet, einen statistisch signifikanten Effekt auf die Verkaufszahlen hat.

Aber nicht zu früh freuen... Ein t-test hat - ähnlich wie die Pearson Korrelation zuvor - ebenfalls voraussetzungen die wir prüfen sollten bevor wie ihn ausführen oder das Ergebnis akzeptieren.

## Voraussetzungen eines independent samples t-tests
Ähnlich wie eine Pearson Korrelation ist ein t-test nur dann wirklich aussagekräftig, wenn die zugrundeliegenden Daten bestimmte Voraussetzungen erfüllen:

1) Das gemessene Sample muss repräsentativ für die Bevölkerung sein, über die wir eine Aussage treffen wollen: (✅) → Wir wissen es hier nicht besser

2) Die Zielvariable muss numerisch sein: ✅

3) Die Beobachtungen sollten unabhängig sein (zwischen Gruppen oder zwischen Teilnehmern wiederholter Beobachtungen): (✅) → Wir wissen es hier nicht besser

4) Freiheit von Ausreißern: ❓ → Müssen wir prüfen

5) Die Zielvariable sollte normalverteilt sein: ❓ → Müssen wir prüfen

6) Homogenität der Varianzen. Beide Verteilungen sollten in etwa die gleiche Varianz haben: ❓ → Müssen wir prüfen



### Ausreißertest

Wir können einen Boxplot zeichnen. Hier wirkt es als hätten wir einige Outlier nach oben und unten.

In [ ]:
df.boxplot(column="Sales_After", by="Group")

Betrachten wir aber diese Datenpunkte im Kontext der Verkaufsverhaltens vor der Intervention wirken die vermeintlichen Ausreißer stimmig. Das scheinen Personen zu sein, die einfahc ingesamt viel oder ingesamt wenig Geld ausgeben. Somit ist es nich verwunderlich, dass sie nach der Intervention außerhalb der Mehrheit liegen.

Ich sehe keinen Grund zu handeln. Höchstens die zwei ganz entfernten Punkte könnten wir ggf. näher untersuchen.

In [ ]:
color_map = {"Control": "grey", "Treatment": "red"}
colors = df["Group"].map(color_map)

df.plot.scatter(x="Sales_Before", y="Sales_After", c=colors)
plt.title("Ausgaben vorher und nachher für Kontrollgruppe (grau) und Interventionsgruppe (rot)")

### Normalitätstest
Normalitätstests kennen wir schon von der Pearson Korrelation.

Bei visueller Inspektion wirken die beiden Variablen normalverteilt. Da der t-test inbesondere bei großen Datenmengen robust gegen leichte Abweichungen von der Normalverteilung ist, genügt uns zunächst dieses Bild.

In [ ]:
df[df.Group == "Control"].Sales_After.hist(alpha=0.5)
df[df.Group == "Treatment"].Sales_After.hist(alpha=0.5)

Eine weitere Methode, die Nähe zur Normalverteilung visuell zu prüfen ist der Q-Q plot, der die tatsächlich beobachteten Quantile mit den Quantilen der theoretisch perfekten (Normal)Verteilung gegenüberstellt. Je näher die Punkte auf der Diagonalen liegen desto näher ist die beobachtete Verteilung an der theoretisch idealen Verteilung.

In unserem Fall sehen wir eine sehr deutliche Annäherung an die Normalverteilung.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.graphics.gofplots import qqplot

# Create Q-Q plot
qqplot(df[df.Group == "Control"].Sales_After, line="s")
plt.title("Q-Q Plot (Control)")
plt.show()

qqplot(df[df.Group == "Treatment"].Sales_After, line="s")
plt.title("Q-Q Plot (Treatment)")
plt.show()


### Homogenität der Varianzen
Ein independent samples t-test ist nur dann richtig aussagekräftig, wenn die jeweiligen Varianzen der zu prüfenden Verteilungen in etwa ähnlich sind.

Einen ersten Eindruck davon gibt uns der Boxplot, den wir oben bereits erstellt haben.

Empirischer können wir das mit einem **Levene Test** prüfen.

In [ ]:
from scipy.stats import levene

stat, p = levene(df[df.Group == "Control"].Sales_After, df[df.Group == "Treatment"].Sales_After)
print(f"Levene’s test statistic: {stat}, p-value: {p}")

# Interpretation
if p > 0.05:
    print("Equal variances assumed (p > 0.05)")
else:
    print("Equal variances not assumed (p <= 0.05)")


Der Test sagt uns, dass die Varianzen empirisch betrachtet nicht homogen sind. Wir sollten also vorsichtig mit der Intepretation unserer t-test Ergebnisse sein, und ggf. einen anderen Test vorziehen.

# Jetzt Du!
**Versuche die gleichen Schritte zu durchlaufen um den Unterschied zwischen den Gruppen in `Customer_Satisfaction_After` zu prüfen.**

# Bonus: Chi-Squared Test
Möchtest du noch tiefer in statistische Verfahren einsteigen? Dann versuche dich mit Hilfe eines KI-Assistenten oder anderer Online-Ressourcen in den  Chi-Squared Test einzuarbeiten, um zu prüfen ob die Intervention erfolgreich dabei war die Verkäufe zu steigern (`Purchase_Made`)

Hinweis: Wir brauchen einen Chi-Squared Test weil die Zielvariable kategorisch (binär) ist und nicht numerisch.